In [1]:
import pandas as pd
import numpy as np
import time
import tqdm
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch_geometric
from torch_geometric.data import download_url, extract_zip, HeteroData
import torch_geometric.transforms as T

In [2]:
seasons = [18, 19, 20, 21, 22]
leagues = {
    "B1": ("Belgian Jupiler Pro League", 37, "Belgian-First-Division"),
    "D1": ("German 1. Bundesliga", 20, "Bundesliga"),
    "D2": ("German 2. Bundesliga", 33, "2-Bundesliga"),
    "E1": ("English Premier League", 9, "Premier-League"),
    "E2": ("English League Championship", 10, "Championship"),
    "F1": ("French Ligue 1", 13, "Ligue-1"),
    "F2": ("French Ligue 2", 60, "Ligue-2"),
    "I1": ("Italian Serie A", 11, "Serie-A"),
    "N1": ("Holland Eredivisie", 23, "Eredivisie"),
    "P1": ("Portuguese Liga ZON SAGRES", 32, "Primeira-Liga"),
    #"SC1": ("Scottish Premiership", 40, "Scottish-Premiership"),
    "SP1": ("Spain Primera Division", 12, "La-Liga"),
    "SP2": ("Spanish Segunda División", 17, "Segunda-Division"),
    "T1": ("Turkish Süper Lig", 26, "Super-Lig"),
}
leagues_df = pd.DataFrame(leagues).transpose().rename(columns={0: "FIFA League Name", 1: "FBREF League Code", 2: "FBREF League Name"})

## Load and prepare player data

In [3]:
# Load and consolidate individual player datasets
player_data = []

for s in seasons:
    data = pd.read_csv(f"player_data/players_{s}.csv", low_memory=False)
    data["season"] = s
    player_data.append(data)

player_data_df = pd.concat(player_data, axis=0).reset_index(drop=True)

# Drop players associated with clubs not participating in leagues in scope
player_data_df = player_data_df[player_data_df["league_name"].isin(list(leagues_df["FIFA League Name"]))]
player_data_df = player_data_df.reset_index(drop=True)

club_data_df = player_data_df[["season", "club_name", "league_name", "league_level"]].copy()
club_data_df = club_data_df.drop_duplicates()


In [4]:
# Prepare data for training

# Remove columns not needed for training
rem_cols = ["player_url", "short_name", "long_name", "dob", "club_loaned_from", "club_jersey_number",
            "club_joined", "club_contract_valid_until", "nationality_id", "nationality_name", "nation_team_id",
            "nation_position", "nation_jersey_number", "body_type", "real_face", "release_clause_eur",
            "player_face_url", "club_logo_url", "club_flag_url", "nation_logo_url", "nation_flag_url",
           "club_team_id", "league_name", "league_level"]
player_data_df = player_data_df.drop(columns=rem_cols)

# Fill nan values
fill_values = {
    "value_eur": 0,
    "pace": 0,
    "shooting": 0,
    "passing": 0,
    "dribbling": 0,
    "defending": 0,
    "physic": 0,
    "mentality_composure": player_data_df["mentality_composure"].mean(),
    "goalkeeping_speed": 0,
}
player_data_df = player_data_df.fillna(value=fill_values)

# Make values numeric
lineup_cols = ["ls", "st", "rs", "lw", "lf", "cf", "rf", "rw", "lam", "cam", "ram", "lm", "lcm", "cm", "rcm", 
               "rm", "lwb", "ldm", "cdm", "rdm", "rwb", "lb", "lcb", "cb", "rcb", "rb", "gk"]
for col in lineup_cols:
    player_data_df[col] = player_data_df[col].apply(lambda x: eval(x))
    
# Translate ordinal categorical values
scale = ["Low", "Medium", "High"]
player_data_df["attacking_work_rate"] = player_data_df["work_rate"].apply(lambda x: scale.index(x.split("/")[0]))
player_data_df["defensive_work_rate"] = player_data_df["work_rate"].apply(lambda x: scale.index(x.split("/")[1]))
player_data_df = player_data_df.drop(columns=["work_rate"])


In [5]:
# Categorize columns for scaling
text_cols = ["sofifa_id", "player_positions", "club_name", "club_position", "preferred_foot", "player_tags", "player_traits", "season"]
num_cols = [x for x in player_data_df.columns if x not in text_cols]

# Scale numerical values
scaler = MinMaxScaler()
player_data_df[num_cols] = scaler.fit_transform(player_data_df[num_cols])


In [6]:
# Create one-hot encodings
list_cols = ["player_positions", "player_tags", "player_traits"]
dummy_cols = []

for col in list_cols:
    player_data_df[col] = player_data_df[col].apply(lambda x: "" if x is np.nan else x)
    data = player_data_df[col].apply(lambda x: str(x).split(","))
    data = data.explode()
    data = pd.get_dummies(data=data, prefix=col)
    data = data.groupby(level=0).max()
    dummy_cols.append(data)
    
dummy_df = pd.concat(dummy_cols, axis=1)

player_data_df = pd.concat([player_data_df, dummy_df], axis=1)
player_data_df = player_data_df = player_data_df.drop(columns=list_cols)

player_data_df = pd.get_dummies(data=player_data_df, columns=["preferred_foot"])


In [7]:
player_data_df.head()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,club_position,...,player_traits_Set Play Specialist,player_traits_Solid Player,player_traits_Speed Dribbler (AI),player_traits_Takes Finesse Free Kicks,player_traits_Target Forward,player_traits_Team Player,player_traits_Technical Dribbler (AI),player_traits_Through Ball,preferred_foot_Left,preferred_foot_Right
0,20801,1.000000,0.979592,0.492268,1.000000,0.615385,0.590909,0.508197,Real Madrid CF,LW,...,0,0,0,0,0,0,0,0,0,1
1,158023,0.979167,0.959184,0.541237,1.000000,0.538462,0.250000,0.377049,FC Barcelona,RW,...,0,0,0,0,0,0,0,0,1,0
2,167495,0.958333,0.938776,0.314433,0.390775,0.576923,0.772727,0.704918,FC Bayern München,GK,...,0,0,0,0,0,0,0,0,0,1
3,176580,0.958333,0.938776,0.500000,0.869452,0.538462,0.522727,0.606557,FC Barcelona,ST,...,0,0,0,0,0,0,0,0,0,1
4,190871,0.958333,0.979592,0.634021,0.477807,0.346154,0.363636,0.311475,Paris Saint-Germain,LW,...,0,0,0,0,0,0,0,0,0,1


In [8]:
player_data_df.to_csv("player_data/player_data.csv", index=False)

## Load and prepare match data

In [9]:
# Load and consolidate individual season datasets
match_data = []

with open('club_data/club_dict.pkl', 'rb') as f:
    club_dict = pickle.load(f)
with open('club_data/club_dict_FBREF.pkl', 'rb') as f:
    club_dict_FBREF = pickle.load(f)

for l in leagues:
    for s in seasons:
        data = pd.read_csv(f"match_data/{l}_{s}.csv", low_memory=False, encoding='windows-1252')
        data = data[~data["HomeTeam"].isnull()]
        data["Season"] = s
        data["Div"] = l
        data["Week"] = 0
        # Replace team names with values used in FIFA dataset
        data["HomeTeam"] = data["HomeTeam"].apply(lambda x: club_dict[x])
        data["AwayTeam"] = data["AwayTeam"].apply(lambda x: club_dict[x])
        
        data_FBREF = pd.read_csv(f"match_data/FBREF/{l}_{s}.csv", low_memory=False)
        if "Round" in data_FBREF.columns:
            data_FBREF = data_FBREF[data_FBREF["Round"] == "Regular season"]
        data_FBREF = data_FBREF[~data_FBREF["Score"].isnull()]
        # Replace team names with values used in FIFA dataset
        data_FBREF["Home"] = data_FBREF["Home"].apply(lambda x: club_dict_FBREF[x])
        data_FBREF["Away"] = data_FBREF["Away"].apply(lambda x: club_dict_FBREF[x])
        
        for row in range(data.shape[0]):
            HT, AT = data["HomeTeam"].iloc[row], data["AwayTeam"].iloc[row]
            HG, AG = data["FTHG"].iloc[row], data["FTAG"].iloc[row]
            score = str(HG) + "–" + str(AG)
            assert data_FBREF[(data_FBREF["Home"] == HT) & (data_FBREF["Away"] == AT)]["Score"].size == 1
            assert data_FBREF[(data_FBREF["Home"] == HT) & (data_FBREF["Away"] == AT)]["Score"].iloc[0] == score
            week = data_FBREF[(data_FBREF["Home"] == HT) & (data_FBREF["Away"] == AT)]["Wk"]
            data.at[row, "Week"] = week
        match_data.append(data)

match_data_df = pd.concat(match_data, axis=0)
match_data_df = match_data_df.reset_index(drop=True)

In [10]:
# Find odds
match_data_df["AvgOdds_H"] = match_data_df[["B365H", "BWH", "IWH", "PSH", "WHH", "VCH"]].mean(axis=1)
match_data_df["AvgOdds_D"] = match_data_df[["B365D", "BWD", "IWD", "PSD", "WHD", "VCD"]].mean(axis=1)
match_data_df["AvgOdds_A"] = match_data_df[["B365A", "BWA", "IWA", "PSA", "WHA", "VCA"]].mean(axis=1)
match_data_df["MaxOdds_H"] = match_data_df[["B365H", "BWH", "IWH", "PSH", "WHH", "VCH"]].max(axis=1)
match_data_df["MaxOdds_D"] = match_data_df[["B365D", "BWD", "IWD", "PSD", "WHD", "VCD"]].max(axis=1)
match_data_df["MaxOdds_A"] = match_data_df[["B365A", "BWA", "IWA", "PSA", "WHA", "VCA"]].max(axis=1)

# Fill nan values
teams = match_data_df["HomeTeam"].unique()
fill_cols = ["HS", "AS", "HST", "AST", "HC", "AC", "HY", "AY", "HR", "AR"]
for t in teams:
    for s in seasons:
        if match_data_df.loc[(match_data_df["HomeTeam"]==t) & (match_data_df["Season"]==s)].shape[0] > 0:
            for col in fill_cols:
                if match_data_df.loc[(match_data_df["HomeTeam"]==t) & (match_data_df["Season"]==s)][col].isnull().sum() > 0:
                    fill_value = {col: match_data_df.loc[(match_data_df["HomeTeam"]==t) & (match_data_df["Season"]==s)][col].mean()}
                    match_data_df.loc[(match_data_df["HomeTeam"]==t) & (match_data_df["Season"]==s)] = match_data_df.loc[(match_data_df["HomeTeam"]==t) & (match_data_df["Season"]==s)].fillna(value=fill_value)


In [11]:
# Keep only relevant columns
cols = ["Div", "Season", "Week", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HS", "AS", "HST", "AST", "HC", "AC",
        "HY", "AY", "HR", "AR", "AvgOdds_H", "AvgOdds_D", "AvgOdds_A", "MaxOdds_H", "MaxOdds_D", "MaxOdds_A"]
match_data_df = match_data_df[cols]

# Categorize columns for scaling
num_cols = ["HS", "AS", "HST", "AST", "HC", "AC", "HY", "AY", "HR", "AR"]

# Scale numerical values
scaler = MinMaxScaler()
match_data_df[num_cols] = scaler.fit_transform(match_data_df[num_cols])

In [12]:
match_data_df.head()

,Div,Season,Week,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,HY,AY,HR,AR,AvgOdds_H,AvgOdds_D,AvgOdds_A,MaxOdds_H,MaxOdds_D,MaxOdds_A
0,B1,18,1,Royal Antwerp FC,RSC Anderlecht,0,0,D,0.065217,0.288889,...,0.3,0.000000,0.0,0.000000,5.301667,3.865000,1.645000,6.11,4.14,1.70
1,B1,18,1,Royal Charleroi S.C.,KV Kortrijk,1,0,H,0.130435,0.200000,...,0.2,0.111111,0.0,0.000000,1.643333,3.786667,5.345000,1.67,4.02,5.62
2,B1,18,1,KAS Eupen,SV Zulte Waregem,0,5,A,0.065217,0.288889,...,0.2,0.222222,0.0,0.000000,2.653333,3.408333,2.573333,2.74,3.55,2.64
3,B1,18,1,KRC Genk,Waasland-Beveren,3,3,D,0.239130,0.200000,...,0.1,0.222222,0.0,0.000000,1.395000,4.695000,7.751667,1.42,5.17,8.96
4,B1,18,1,Sporting Lokeren,Club Brugge KV,0,4,A,0.239130,0.200000,...,0.3,0.111111,0.0,0.111111,4.246667,3.630000,1.821667,4.33,3.78,1.89


In [13]:
match_data_df.to_csv("match_data/match_data.csv", index=False)